In [ ]:
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG ATGTGTAGCGA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG ATGTGTAGCGA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GACCATCCCCT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG CGTATCCCGAA CTGACGCGCACGAATTACAGATAGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AAACCGAAGGT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG CGCCCAATCTA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA
CCTTCAATTTCGATGGGTACCACCCAGCTTTCCTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AAACCGAAGGT CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAACAA


#from slow R1

CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GAGATCTCAAA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG TGTTAAGGCCG CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG AATAGCACTGC CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG
CCTTCAATTTCGATGGGTACCACCCAGCTTTCTTGTACAAAGTGGTTGATATCTCTATAGTCGCAGTAGGCGG GAGATCTCAAA CTGACGCGCACGAATTACAGATCGGAAGAGCGGTTCAGCAGGAATGCCGAGACCGATCTCAGAAATG


In [110]:
#pd.read_csv("/corgi/otherdataset/ellenbushell/P28011/sampleinfo.txt",sep="\t")
#bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/P28011/bc.csv",sep=" ")


import gzip
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import regex as re

def open_gzip_optional(fname,mode="rt"):
    if str(fname).endswith("gz"):
        return gzip.open(fname,mode)
    elif str(fname).endswith("bz2"):
        return bz2.open(fname,mode)
    else:
        return open(fname,mode)
            
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])

rootdir = Path("/corgi/otherdataset/ellenbushell/june_pools_P28011/")
fastqdir = rootdir / "fastq"

################ Read barcode collection
bc_file = pd.read_csv("/home/pcddas/BARSEQ_RAW_COUNTS_AND_METADATA/barcode_to_gene_210920_FRANK.csv")
bc_file["mutant"]=[x.split("|")[0] for x in bc_file["gene"]]
bc_file["barcode"] = [x.upper() for x in bc_file["barcode"]]
dict_bc2mutant = dict(zip(bc_file["barcode"],bc_file["mutant"]))

################ Prepare parser
flank_regex = re.compile("GGCGG"+"(\w{8,16})"+"CTGAC")

################ Process all files
#sample_dict = {}
list_samples = []
list_counts = []
for seq_file in fastqdir.iterdir():
    if str(seq_file).endswith("fastq.gz"):
        
        #if len(list_samples)>5:
        #    break


        ####### Extract sample name
        #P28011_1172_S11_L001_R1_001.fastq.gz
        # => P28011_1172
        #sample = format_filename(seq_file)
        sample = seq_file.name.split("_S")[0]
        
        #Empty counts dictionary
        count_dict = dict(zip(list(bc_file["mutant"]), [0]*bc_file.shape[0]))
        count_dict["_other"] = 0 
        count_dict["_err"] = 0 
        
        #Reverse complement if read2
        do_revcomp = !("_R2_" in str(seq_file))
        
        if "_R1_" in seq_file.name:
            sample = sample+":R1"
        else:
            sample = sample+":R2"
        print(sample)
        
        
        #For all lines
        with open_gzip_optional(seq_file) as f1:
            it1 = iter(f1)
            count = 0
            while True:
                try:
                    
                    #Read one entry in FASTQ
                    next(it1)
                    theseq = next(it1).strip()
                    next(it1)
                    next(it1)
                    
                    if do_revcomp:
                        theseq = reverse_complement(theseq)

                    #print("> "+theseq)
                        
                    #Parse one entry
                    try:
                        
                        putative_barcode = re.search(flank_regex, theseq)[1]
                        #print(putative_barcode)
                        
                        putative_barcode = reverse_complement(putative_barcode)  #for this input file
                        
                        if putative_barcode in dict_bc2mutant.keys():
                            count_dict[dict_bc2mutant[putative_barcode]] += 1
                        else:
                            count_dict["_other"] += 1
                            #print("not in dict "+putative_barcode)

                    except TypeError:
                        count_dict["_err"] += 1
                        #print("type err: "+theseq)
                    
                    count=count+1

                except StopIteration:
                    break
        
        #Turn into an array, add to the list
        list_samples.append(sample)
        list_counts.append(
            np.array([count_dict["_other"], count_dict["_err"]]+[count_dict[i] for i in bc_file["mutant"]])
        )
        
        

### Turn it all into one matrix
df = pd.DataFrame(
    np.vstack(list_counts).T,
    columns = list_samples, 
    index = ["_other","_err"]+list(bc_file["mutant"]))

P28011_1009:R2
P28011_1010:R2
P28011_1016:R1
P28011_1017:R2
P28011_1023:R1
P28011_1032:R1
P28011_1034:R1
P28011_1034:R2
P28011_1035:R1
P28011_1043:R2
P28011_1045:R1
P28011_1052:R1
P28011_1052:R2
P28011_1055:R2
P28011_1056:R2
P28011_1057:R2
P28011_1064:R2
P28011_1068:R2
P28011_1070:R2
P28011_1081:R2
P28011_1088:R1
P28011_1092:R1
P28011_1095:R2
P28011_1096:R2
P28011_1099:R2
P28011_1101:R2
P28011_1102:R1
P28011_1102:R2
P28011_1106:R2
P28011_1107:R1
P28011_1108:R1
P28011_1108:R2
P28011_1109:R1
P28011_1112:R1
P28011_1114:R1
P28011_1117:R1
P28011_1121:R2
P28011_1123:R2
P28011_1125:R1
P28011_1130:R1
P28011_1130:R2
P28011_1136:R1
P28011_1138:R1
P28011_1153:R1
P28011_1157:R1
P28011_1159:R1
P28011_1161:R1
P28011_1026:R1
P28011_1045:R2
P28011_1020:R2
P28011_1030:R1
P28011_1023:R2
P28011_1004:R2
P28011_1020:R1
P28011_1032:R2
P28011_1036:R1
P28011_1037:R2
P28011_1046:R2
P28011_1024:R1
P28011_1022:R1
P28011_1036:R2
P28011_1012:R1
P28011_1008:R1
P28011_1031:R2
P28011_1025:R1
P28011_1013:R2
P28011_102

In [111]:
df

,P28011_1009:R2,P28011_1010:R2,P28011_1016:R1,P28011_1017:R2,P28011_1023:R1,P28011_1032:R1,P28011_1034:R1,P28011_1034:R2,P28011_1035:R1,P28011_1043:R2,...,P28011_1127:R2,P28011_1069:R1,P28011_1008:R2,P28011_1006:R2,P28011_1043:R1,P28011_1038:R2,P28011_1072:R1,P28011_1147:R2,P28011_1056:R1,P28011_1135:R1
_other,1311,2664,0,759,0,0,0,1957,0,1311,...,2493,0,118,5371,0,697,0,1012,0,0
_err,1592,2732,27662,637,42667,43635,29994,1046,30777,1381,...,1674,55115,1335,26601,47186,593,110957,782,25091,112385
PBANKA_093210,0,1,0,0,0,0,0,3,0,2,...,0,0,0,13,0,0,0,0,0,0
PBANKA_142090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_031260,65,169,0,37,0,0,0,49,0,81,...,92,0,5,80,0,50,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PBANKA_071030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_021280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_051730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PBANKA_010440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
#Check how well it worked
df[0:2]/df.sum(axis=0)




,P28011_1009:R2,P28011_1010:R2,P28011_1016:R1,P28011_1017:R2,P28011_1023:R1,P28011_1032:R1,P28011_1034:R1,P28011_1034:R2,P28011_1035:R1,P28011_1043:R2,...,P28011_1127:R2,P28011_1069:R1,P28011_1008:R2,P28011_1006:R2,P28011_1043:R1,P28011_1038:R2,P28011_1072:R1,P28011_1147:R2,P28011_1056:R1,P28011_1135:R1
_other,0.026658,0.028801,0.0,0.028643,0.0,0.0,0.0,0.064407,0.0,0.027500,...,0.041087,0.000000,0.024671,0.061726,0.0,0.027588,0.0,0.013993,0.0,0.0
_err,0.032372,0.029536,1.0,0.024039,1.0,1.0,1.0,0.034425,1.0,0.028969,...,0.027589,0.999982,0.279114,0.305709,1.0,0.023471,1.0,0.010813,1.0,1.0


In [113]:
df.to_csv(rootdir / "counts.csv")

#rootdir = Path("/corgi/otherdataset/ellenbushell/P28011/")
